In [1]:
import math
import numpy as np
from config import *
import toolbox as tb
from debug_casadi import *
from evaluate import create_visualization
import scipy.linalg as sl

ITER_TIMES = 15
DRAWPT_PER_PIECE = 20

def constructBetaT(t, rank:int):
    ''' 构造特定时间的β(t) '''
    beta = np.zeros((NCOFF, 1), dtype=np.float64)
    for i in range(rank, NCOFF):
        if not isinstance(t, int|float) or t!=0 or i-rank==0:
            beta[i,0] = math.factorial(i)/math.factorial(i-rank) * t**(i-rank)
    return beta

def constructMincoM(pieceT):
    mat_m = np.zeros((NCOFF, NCOFF), dtype=np.float64)
    for i in range(NCOFF-1):
        mat_m[i, :] = constructBetaT(0, i).T

    mat_m[-1, :] = constructBetaT(pieceT, 0).T
    return mat_m

def constructMincoQ(last_coff, tgtPos, pieceT):
    mat_r = consturctMatR(pieceT=pieceT)
    mat_q = mat_r @ last_coff
    mat_q[-1, :] = tgtPos
    return mat_q

def consturctMatR(pieceT):
    mat_r = np.zeros((NCOFF, NCOFF), dtype=np.float64)
    for i in range(NCOFF):
        mat_r[i, :] = constructBetaT(pieceT, i).T
    return mat_r

def constructInitialCoff(init_pos):
    coff = np.zeros((NCOFF, NDIM), dtype=np.float64)
    coff[0,:] = init_pos
    return coff
    # init_state = ca.SX.sym('init_state', NCOFF, NDIM) # type: ignore


In [2]:

pieceT = 1

mat_m_inv = np.linalg.inv(constructMincoM(pieceT))
mat_r = consturctMatR(pieceT=pieceT)
mat_s = np.diag([1,1,1,1,1,0])
mat_u = np.array([[0,0,0,0,0,1]]).T

mat_F = mat_m_inv @ mat_s @ mat_r               # matF是行向量
mat_G = (mat_m_inv @ mat_u).reshape(-1,1)       # matG是列向量

In [8]:
ccm = np.zeros((6,6))#np.array([mat_G, mat_F@mat_G, mat_F@mat_F@mat_G])
ntimes_F_pow = np.identity(6)
for i in range(6):
    ccm[:,i] = (ntimes_F_pow @ mat_G).reshape(-1)
    ntimes_F_pow = mat_F @ ntimes_F_pow
# ccm
np.linalg.matrix_rank(ccm)

6

In [11]:
import control

def constructBBTint(pieceT, rank):
    ''' c^T*(∫β*β^T*dt)*c ==> (2, NCOFF) * (NCOFF, NCOFF) * (NCOFF, 2) '''
    bbint = np.zeros((NCOFF, NCOFF))
    beta = constructBetaT(pieceT, rank)
    for i in range(NCOFF):
        for j in range(NCOFF):
            if i+j-2*rank < 0: continue
            coff = 1 / (i+j-2*rank+1)
            bbint[i, j] = coff * beta[i,0] * beta[j,0] * pieceT
    return bbint

# 定义权重矩阵
Q = constructBBTint(pieceT=pieceT, rank=3)
R = np.array([[1]])     # 控制权重矩阵

K, S, E = control.dlqr(mat_F, mat_G, Q, R)



array([[ -0.9954253 ,  -1.98021064,  -3.91439464,  -7.63946461,
        -14.56728764, -26.68011065]])